https://platform.openai.com/docs/guides/function-calling 

`We recommend that you use no more than 20 tools in a single API call.`

`Developers typically see a reduction in the model's ability to select the correct tool once they have between 10-20 tools defined.`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from openai import OpenAI

client = OpenAI()

In [ ]:
from datetime import datetime

def get_delivery_date(order_id: str):
    # here will be really compilacted function logic
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
get_delivery_date_tool = {
    "type": "function",
    "function": {
        "name": "get_delivery_date",  # it is good to use real function name here
        "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The customer's order ID.",
                },
            },
            "required": ["order_id"],
            "additionalProperties": False,
        },
    }
}

In [ ]:
def get_weather(location: str, unit: str):
    # e.g. may call external API
    return "Sunny :) 23 C."

In [ ]:
get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "Location to check weather in."},
                "unit": {"type": "string", "enum": ["C", "F"], "description": "Unit of temperature: C - Celsius, F - Fahrenheit"},
            },
            "required": ["location", "unit"],
            "additionalProperties": False,
        },
    },
}

In [ ]:
funcs_to_call = {
    "get_weather": get_weather,
    "get_delivery_date": get_delivery_date,
}

In [ ]:
tools = [get_weather_tool, get_delivery_date_tool]

messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "Hi, can you tell me the delivery date for my order?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response.model_dump()

In [ ]:
messages.append({"role": "assistant", "content": response.choices[0].message.content})
messages.append({"role": "user", "content": "It is 902834-235-gdf"})

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response.model_dump()

In [ ]:
import json

tool_call = response.choices[0].message.tool_calls[0].model_dump()
arguments = json.loads(tool_call['function']['arguments'])
function_name = tool_call['function']['name']

In [ ]:
function_name, arguments

In [ ]:
result = funcs_to_call[function_name](**arguments)
result

In [ ]:
function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        **arguments,
        "result": result
    }),
    "tool_call_id": response.model_dump()['choices'][0]['message']['tool_calls'][0]['id']
}
function_call_result_message

In [ ]:
messages.append(response.choices[0].message)  # func call
messages.append(function_call_result_message)  # function result

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [ ]:
response.choices[0].message.role, response.choices[0].message.content